# Bedrock Prompt Examples for Travel

In this notebook, we include different example use cases for Travel & Hospitality using Amazon Bedrock.

These sample use cases involve different tasks and prompt engeering techniques, as follows:
1. **Generate recommendations based on metadata**
    - **Task:** Text Generation
    - **Prompt Technique:** Zero-shot
2. **Estimate capacity for airlines or hotel properties based on historical data**
    - **Task:** Complex Reasoning
    - **Prompt Technique:** Chain-of-Thoughts (CoT)
3. **Create a question answering assistant for customer service**
    - **Task:** Question Answering with Dialogue Asistant (without memory)
    - **Prompt Technique:** Few-shot
4. **Summarize and classify content from media files transcription**
    - **Task:** Text Summarization & Text Classification
    - **Prompt Technique:** Zero-shot
5. **Create splash pages describing upcoming promotions**
    - **Task:** Code Generation
    - **Prompt Technique:** Zero-shot

Let's start by ensuring the Bedrock SDK is properly installed.

We'll also install a few libraries required in the notebook.

In [ ]:
#!unzip ../bedrock-preview-documentation/SDK/bedrock-python-sdk.zip -d /root/bedrock

#!pip install --upgrade pip
#!pip install scikit-learn seaborn

#!pwd
#!python3 -m pip install /root/bedrock/boto3-1.26.142-py3-none-any.whl
#!python3 -m pip install /root/bedrock/botocore-1.29.142-py3-none-any.whl

Now we can import the libraries and setup the Bedrock client.

In [5]:
import boto3
import json
import csv
from datetime import datetime

bedrock = boto3.client(
 service_name='bedrock',
 region_name='us-east-1',
 endpoint_url='https://bedrock.us-east-1.amazonaws.com'
)

Let's get the list of Foundational Models supported in Bedrock at this time.

In [5]:
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': 'f0d858da-ce48-42af-9236-79d0aab293f8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 27 Jun 2023 11:11:50 GMT',
   'content-type': 'application/json',
   'content-length': '861',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f0d858da-ce48-42af-9236-79d0aab293f8'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-e1t-medium',
   'modelId': 'amazon.titan-e1t-medium'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/stability.stable-diffusion-xl',
   'modelId': 'stability.stable-diffusion-xl'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-grande-instruct',
   'modelId': 'ai21.j2-grande-instruct'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-jumbo-instruct',
   'modelId': 'ai21.j2-jumbo-in

We will define an utility function for calling Bedrock.

This will help passing the proper body depending on the model invoked, and will store the results in a CSV file as well.

In [6]:
def call_bedrock(modelId, prompt_data):
    if 'amazon' in modelId:
        body = json.dumps({
            "inputText": prompt_data,
            "textGenerationConfig":
            {
                "maxTokenCount":4096,
                "stopSequences":[],
                "temperature":0,
                "topP":0.9
            }
        })
        #modelId = 'amazon.titan-tg1-large'
    elif 'anthropic' in modelId:
        body = json.dumps({
            "prompt": prompt_data,
            "max_tokens_to_sample": 4096,
            "stop_sequences":[],
            "temperature":0,
            "top_p":0.9
        })
        #modelId = 'anthropic.claude-instant-v1'
    elif 'ai21' in modelId:
        body = json.dumps({
            "prompt": prompt_data,
            "maxTokens":4096,
            "stopSequences":[],
            "temperature":0,
            "topP":0.9
        })
        #modelId = 'ai21.j2-grande-instruct'
    elif 'stability' in modelId:
        body = json.dumps({"text_prompts":[{"text":prompt_data}]}) 
        #modelId = 'stability.stable-diffusion-xl'
    else:
        print('Parameter model must be one of titan, claude, j2, or sd')
        return
    accept = 'application/json'
    contentType = 'application/json'

    before = datetime.now()
    response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    latency = (datetime.now() - before)
    response_body = json.loads(response.get('body').read())

    if 'amazon' in modelId:
        response = response_body.get('results')[0].get('outputText')
    elif 'anthropic' in modelId:
        response = response_body.get('completion')
    elif 'ai21' in modelId:
        response = response_body.get('completions')[0].get('data').get('text')

    #Add interaction to the local CSV file...
    #column_name = ["timestamp", "modelId", "prompt", "response", "latency"] #The name of the columns
    #data = [datetime.now(), modelId, prompt_data, response, latency] #the data
    #with open('./prompt-data/prompt-data.csv', 'a') as f:
    #    writer = csv.writer(f)
    #    #writer.writerow(column_name)
    #    writer.writerow(data)
    
    return response, latency

Now we are ready for running our examples with different models.

-----

## 1. Generate recommendations based on metadata

**Use Case:** A company wants to generate recommendations of flight destinations for their users based on some metadata, e.g. country, age-range, and interests.

**Task:** Text Generation

**Prompt Technique:** Zero-shot

In [7]:
prompt_data ="""
Human:
Generate a list of 10 recommended destinations for traveling considering the information in the <metadata></metadata> XML tags, and include a very brief description of each recommendation.

<metadata>
Passenger country is Spain
Age range between 20-30
Interested on water sports and theme parks
Traveling during the summer
</metadata>

Assistant:
"""

In [8]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)

 Here is a list of 10 recommended destinations for traveling based on the provided metadata:

1. Hawaii - Beautiful beaches, surfing, and vibrant culture. Home to massive water parks and theme parks. 
2. Bali, Indonesia - Exotic island with stunning beaches, jungles, and rice paddies. Plenty of opportunities for snorkeling, boating, and other watersports. Also home to an exciting theme park.
3. Orlando, Florida - Theme park capital of the world, including Walt Disney World and Universal Studios. Also close to beautiful beaches on the Gulf coast. 
4. Costa Rica - Pristine natural scenery, rainforests, and beaches along both the Pacific and Caribbean coasts. Eco-friendly destination for watersports and adventure activities.  
5. Dubai, UAE - Glamorous city in the desert with massive water parks, indoor skiing, and the world's tallest rollercoaster. Also a hub for boating, jet skiing, and other watersports.
6. Queensland, Australia - Miles of beaches along the Great Barrier Reef, perfect 

In [9]:
response, latency = call_bedrock('anthropic.claude-instant-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)

 Here are 10 recommended destinations for traveling:

Dubai - An ultramodern city with amazing architecture, shopping malls, and water parks.  

Maldives - A tropical paradise with beautiful beaches, crystal clear water for snorkeling, and overwater bungalows.

Orlando, Florida - Home to many theme parks like Disney World, Universal Studios, and SeaWorld with roller coasters and water rides.   

Bali, Indonesia - Tropical island known for its beaches, surfing, water sports, and Hindu culture.

Phuket, Thailand - Beautiful beaches, waterfalls, and islands for snorkeling and diving near Phuket's lively city.   

Barcelona, Spain - Home city with beautiful architecture, beaches, and theme parks like PortAventura.

Ibiza, Spain - Famous for nightlife but also has beautiful beaches and coves for swimming and water sports.

Miami, Florida - Beaches, art deco architecture, theme parks like Wet n Wild, and many water activities.   

Cancun, Mexico - Long stretches of white sand beaches, turquo

------

## 2. Estimate capacity for airlines or hotel properties based on historical data

**Use Case:** A T&H company wants to estimate capacity/occupancy levels they could have for the next days based on historical information and flights/hotels metadata.

**Task:** Complex Reasoning

**Prompt Technique:** Chain-of-Thoughts (CoT)

In [7]:
prompt_data ="""
Human: Last week, the passengers of 3 routes of an airline were according to the following data:
- Monday: Paris 650, New York 320, Singapore 415
- Tuesday: Paris 640, New York 330, Singapore 410
- Wednesday: Paris 630, New York 340, Singapore 425

Question: How many passengers can we expect next Friday on the route to Paris?
Answer: According to the numbers given and without having more information, there is a daily decrease of 10 passengers for the route to Paris.
If we assume that this trend will continue during the following days, we can expect 620 passengers for the next day which is Thursday and,
therefore, 610 passengers for Friday.

Question: How many passengers can we expect on Saturday on each of the routes? Reason step by step and provide recommendations to increase the passengers
Assistant:
Answer:
"""

In [8]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)

- For the Paris route: There is a daily decrease of 10 passengers. Since Friday had 610 passengers, we can expect 600 passengers on Saturday.
- For the New York route: There is a daily increase of 10 passengers. Since Friday had 340 passengers, we can expect 350 passengers on Saturday.
- For the Singapore route: There is no clear trend. The numbers go up and down between 410 to 425 passengers. Without more data, the best assumption is to expect around 420 passengers on Saturday.

Recommendations to increase passengers:

1. Offer promotions and discounts: This can attract more customers to choose these routes, especially for Paris and Singapore where the numbers are decreasing or unstable.

2. Improve service quality: Better service and experience can lead to higher customer satisfaction and loyalty. This can translate to repeat customers and word-of-mouth marketing to new potential customers. 

3. Optimize flight schedules: Review the flight schedules and frequencies to ensure they are

In [9]:
response, latency = call_bedrock('anthropic.claude-instant-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)

Based on the given data and trend, we can estimate the number of passengers for Saturday as follows:

Paris route:
- Monday: 650 passengers
- Tuesday: 640 passengers  (a decrease of 10 passengers from Monday)
- Wednesday: 630 passengers (a decrease of 10 passengers from Tuesday)
- Thursday (estimated): 620 passengers (a decrease of 10 passengers from Wednesday)
- Friday (estimated): 610 passengers (a decrease of 10 passengers from Thursday)
- Saturday (estimated): 600 passengers (a decrease of 10 passengers from Friday)

New York route:
- Monday: 320 passengers
- Tuesday: 330 passengers (an increase of 10 passengers from Monday)  
- Wednesday: 340 passengers (an increase of 10 passengers from Tuesday)
- Assuming the same trend continues:
Thursday (estimated): 350 passengers (an increase of 10 passengers from Wednesday)
Friday (estimated): 360 passengers (an increase of 10 passengers from Thursday)  
Saturday (estimated): 370 passengers (an increase of 10 passengers from Friday)

Singap

------

## 3. Create a question answering assistant for customer service

**Use Case:** A company wants to create a bot capable of answering questions about the services available, based on the internal information for these.

**Task:** Question Answering with Dialogue Asistant (no memory)

**Prompt Technique:** Few-shot

In [20]:
prompt_data ="""
Context: An airline services available for purchase are as follows
1. Seat upgrades, available from 20 Euros, to classes Economy Plus and Business, on weekdays' flights
2. Meals, available for payment in-flight, mediterranean menu, on all days' flights
3. Fast boarding, from 30 Euros, available for premier customers, on all days' flights

Instruction: Answer any questions about the services available in a friendly manner. If you don't know the answer just say 'Apologies, but I don't have the answer for that. Please contact our team by phone.'

Assistant: Welcome to Airline Services, how can I help you?
Human: Hi, I would like to know what are the services available please.
Assistant: Of course, right now we have the seat upgrades, the meals, and the fast boarding.
Human: Thank you. I would like to know details for those please.
Assistant:
"""

In [21]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)

 Certainly, here are the details for our available services:

Seat upgrades:
- Economy Plus upgrade: Extra legroom seats, available for 20 Euros on weekdays' flights. 
- Business class upgrade: Wider, more comfortable seats, available for 50-100 Euros depending on the route, on weekdays' flights.

Meals:
- Mediterranean menu: A selection of meals like pasta, pizza, paninis available for around 10 Euros, on all days' flights.

Fast boarding:
- For our premier customers, fast boarding option allows you to board the plane before general boarding, available for 30 Euros on all days' flights.

Does this help explain our services? Let me know if you have any other questions. 

 Inference time: 0:00:04.445540


In [22]:
response, latency = call_bedrock('anthropic.claude-instant-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)

 Here are the details:

1. Seat upgrades, available from 20 Euros, to classes Economy Plus and Business, on weekdays' flights. This allows you to move to a more spacious seat with extra legroom.

2. Meals, available for payment in-flight, mediterranean menu, on all days' flights. You can choose from a selection of meals like pasta, salad, sandwiches, etc. 

3. Fast boarding, from 30 Euros, available for premier customers, on all days' flights. This allows you to board the plane before other passengers so you can get settled in quickly.

Please let me know if you need any clarification or have additional questions. I'm happy to help. 

 Inference time: 0:00:01.706800


------

## 4. Generate content summary based on transcriptions from media files

**Use Case:** A company needs to generate summaries of the audio transcription for audio and video files for customer service, to be sent to their operations quality team. They also want to classify this content according to specific categories.

**Task:** Text Summarization & Text Classification

**Prompt Technique:** Zero-shot

#### (Pre-requisite)

First, we will need to transcribe the media files. You can e.g. use Amazon Transcribe for this task following examples like this: https://docs.aws.amazon.com/code-library/latest/ug/transcribe_example_transcribe_StartTranscriptionJob_section.html

For our sample we will start from an interview transcription in the file "interview.txt".

In [24]:
f = open("interview.txt", "r").read()
print(f)

Jimmy Fallon: (00:00)
Thank you for doing This Christmas Will Be Different. I appreciate you doing that. We hung out all day together.

Matthew McConaughey: (00:04)
How fun was that?

Jimmy Fallon: (00:05)
It was the greatest. We always…

Matthew McConaughey: (00:07)
Two take maximum. Two take maximum.

Jimmy Fallon: (00:09)
We nailed it, we have some good takes. That was so much fun. Happy holidays to you and your family. I’m happy to see you in person. Thanks for making the trip and being here.

Matthew McConaughey: (00:18)
Absolutely.

Jimmy Fallon: (00:19)
Do you have like family traditions? Do you have holiday plans? What do you have coming up?

Matthew McConaughey: (00:24)
This year, we’re going to get all the McConaughey family. We’re also getting Camila’s family. So when we do that and we have that multicultural Christmas, Christmas lasts a long time at our place because they celebrate on the 24th. Midnight you exchange presents. All right. So you have the big sit down dinner, 

In [25]:
prompt_data =f"""
Human:
Generate a summary of the transcription in the <transcription></transcription> XML tags below.
Then, classify the mood of the participants according to the closest category within 'fully satisfied', 'satisfied', 'unsatisfied', 'fully unsatisfied', or 'neutral'.

<transcription>
{f}
</transcription>

Assistant:
"""

In [26]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)

 Here is a summary of the transcription:

Jimmy Fallon and Matthew McConaughey discuss McConaughey's holiday plans, family traditions, and philanthropic work. McConaughey talks about celebrating Christmas with both his and his wife Camila's extended families, which results in long days of present opening and meals. Fallon and McConaughey bond over memories of saving wrapping paper and impatiently waiting to open presents with family. 

McConaughey discusses his decision not to run for Texas governor at this point to focus on his family and storytelling. However, he does not rule out running for political office in the future. McConaughey also talks about organizing a benefit concert that raised over $7 million for Texans affected by Winter Storm Uri. Fallon praises McConaughey for his philanthropic work and showing up to help others in times of need.

Overall, the mood of the participants seems satisfied and content discussing family, charity work, and life events. The friendly, casual

In [27]:
response, latency = call_bedrock('anthropic.claude-instant-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)

 Here is a summary of the transcription:

- Jimmy Fallon and Matthew McConaughey discuss Matthew filming a Christmas song for Jimmy's show. They had fun filming it together.

- Matthew talks about his family's Christmas traditions which involve both his and his wife's large extended families. They celebrate on Christmas Eve and Christmas Day, so the celebrations last a long time. 

- They discuss New Year's resolutions and Matthew says his resolution last year was to work on his patience. 

- Jimmy congratulates Matthew on the success of his book "Green Lights" which was a New York Times bestseller for 50 weeks and sold over 2.5 million copies. Matthew says he had no idea it would be so successful but it hit a nerve with people.

- They discuss Matthew's new companion journal to his book to help people journal their own stories. 

- Matthew considered running for governor of Texas but recently decided not to run, citing his young children and wanting to focus on storytelling at this po

------

## 5. Create splash pages describing upcoming promotions

**Use Case:** A company wants to create HTML pages quickly and easily for their upcoming promotions.

**Task:** Code Generation

**Prompt Technique:** Zero-shot

In [34]:
prompt_data ="""
There is an upcoming promotion presented by the Spanish Wings airline.
The promotion is targeting young audience in the age range between 18 and 40.
The promotion consists of a 20% discount when purchasing tickets online.
There will be additional fees for seat assignment and tickets can be bought trought this same portal.
The promotion is part of the Summer Discounts of the company.
The promotion is available from June 28th to August 31st.

Based the this information, generate the HTML code for an attractive splash page for this promotion.
Include catchy phrases and invite customers to sign-up for the airlines' loyalty program.
Have the splash page use yellow fonts and black background, according to the airlines' branding.
"""

In [35]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data)
#print(response, "\n\n", "Inference time:", latency)
from IPython.display import display, HTML
display(HTML(response))

In [36]:
response, latency = call_bedrock('anthropic.claude-instant-v1', prompt_data)
#print(response, "\n\n", "Inference time:", latency)
from IPython.display import display, HTML
display(HTML(response))

------